In [95]:
import pandas as pd
import re
import datetime

In [96]:
belsimple_path = r'C:\Users\Aniket22\OneDrive\Desktop\Task\coding_task_1\raw_belsimpel.xlsx'
shopee_path = r'C:\Users\Aniket22\OneDrive\Desktop\Task\coding_task_1\raw_shopee_Sg.xlsx'
output_path = r'C:\Users\Aniket22\OneDrive\Desktop\Task\coding_task_1\output_shopee_belsimpel.xlsx'

In [97]:
##Belsimpel


In [98]:
belsimple_df = pd.read_excel(belsimple_path).dropna(subset = ['Review Text', 'Pro_Con Review'], how='all')
fillna_values = {'Review Text' : '', 'Pro_Con Review':''}
belsimple_df.fillna(fillna_values, inplace=True)
shopee_df = pd.read_excel(shopee_path)
output_df = pd.read_excel(output_path)

C:\Users\Aniket22\AppData\Local\Temp\ipykernel_18352\3553394248.py:5: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  output_df = pd.read_excel(output_path)


In [99]:
output_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Country      411 non-null    object        
 1   Source       411 non-null    object        
 2   Rating       411 non-null    int64         
 3   Review Date  411 non-null    datetime64[ns]
 4   Review Text  411 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 16.2+ KB


In [100]:
final_output_df = pd.DataFrame(columns = ['Country', 'Source', 'Rating', 'Review Date', 'Review Text'])

In [101]:
def get_pros_and_cons(review):
    if type(review)==str:
        splitted = review.split('Minpunt')
        cons = splitted[1:]
        pros = splitted[0].split('Pluspunt')
        cons = list(map(str.strip, cons))
        pros = list(map(str.strip, pros))
        return f"'.' Pros: {', '.join(pros[1:])} '.' Cons: {', '.join(cons)}"
    else:
        return ''

In [102]:
def get_datetime(text):
    day = re.search('\d{2}-\d{2}-\d{4}', text)
    return str(datetime.datetime.strptime(day.group(), '%d-%m-%Y'))

In [103]:
belsimple_final_reviews = []
for index, row in belsimple_df.iterrows():
    belsimple_final_reviews.append((row['Review Text'] + ' ' + get_pros_and_cons(row['Pro_Con Review'])).strip())

belsimple_df['Review Text'] = belsimple_final_reviews

In [104]:
#  sanity check
random_row = 99
belsimple_df.iloc[random_row, 4]==output_df['Review Text'][random_row]

True

In [105]:
belsimple_df['Review Date'] = belsimple_df['Review Date'].apply(get_datetime)

In [106]:
belsimple_df['Rating'] = belsimple_df['Rating'].apply(lambda x : int(float((x.split()[0]).strip())))

In [107]:
belsimple_df['Country'] = 'Netherlands'
belsimple_df['Source'] = 'Belsimple'

In [108]:
final_output_df = pd.concat([final_output_df, belsimple_df[['Country', 'Source', 'Rating', 'Review Date', 'Review Text']]], axis=0)

In [109]:
final_output_df.head()

,Country,Source,Rating,Review Date,Review Text
0,Netherlands,Belsimple,5,2022-10-20 00:00:00,Vanaf een Huawei P30 pro die kapot ging de ove...
1,Netherlands,Belsimple,5,2022-10-18 00:00:00,Voor de pure Android liefhebber is het een uit...
2,Netherlands,Belsimple,4,2022-10-17 00:00:00,Na jaren geleden 2x een Nexus gehad te hebben ...
3,Netherlands,Belsimple,5,2022-10-14 00:00:00,"Ik had hiervoor al de Pixel 6, daar was niks m..."
4,Netherlands,Belsimple,4,2022-10-27 00:00:00,"Pixel 7 nu een ruime week in bezit, en beval m..."


In [110]:
##shopee

In [111]:
shopee_df['Rating'] = shopee_df['Rating'].apply(lambda x : x.count('polygon points'))

In [112]:
def get_datetime_shopee_sg(text):
    day = re.search('\d{4}-\d{2}-\d{2}', text)
    return str(datetime.datetime.strptime(day.group(), '%Y-%m-%d'))

In [113]:
shopee_df['Review Date'] = shopee_df['Review Date'].apply(get_datetime_shopee_sg)

In [114]:
def get_review_text_shopee(text):
    text_list = text.split('</div>')
    if len(text_list)==1:
        return text
    final_text_list = []
    
    for text in text_list:
        final_text_list.append(re.compile(r'<[^>]+>').sub('', text))
    return " '.' ".join(final_text_list[:-1])


In [115]:
shopee_df['Review Text'] = shopee_df['Review Text'].apply(get_review_text_shopee)

In [116]:
#  sanity check
random_row = 7
shopee_df['Review Text'][random_row]==output_df['Review Text'][381+random_row]

True

In [117]:
shopee_df['Country'] = 'Singapore'
shopee_df['Source'] = 'Shopee-SG'

In [118]:
final_output_df = pd.concat([final_output_df, shopee_df[['Country', 'Source', 'Rating', 'Review Date', 'Review Text']]], axis=0)

In [119]:
final_output_df

,Country,Source,Rating,Review Date,Review Text
0,Netherlands,Belsimple,5,2022-10-20 00:00:00,Vanaf een Huawei P30 pro die kapot ging de ove...
1,Netherlands,Belsimple,5,2022-10-18 00:00:00,Voor de pure Android liefhebber is het een uit...
2,Netherlands,Belsimple,4,2022-10-17 00:00:00,Na jaren geleden 2x een Nexus gehad te hebben ...
3,Netherlands,Belsimple,5,2022-10-14 00:00:00,"Ik had hiervoor al de Pixel 6, daar was niks m..."
4,Netherlands,Belsimple,4,2022-10-27 00:00:00,"Pixel 7 nu een ruime week in bezit, en beval m..."
...,...,...,...,...,...
25,Singapore,Shopee-SG,5,2022-09-22 00:00:00,Performance: should be good '.' Best Feature(s...
26,Singapore,Shopee-SG,5,2022-10-10 00:00:00,Performance: GREAT '.' Best Feature(s): color ...
27,Singapore,Shopee-SG,5,2022-10-15 00:00:00,Delivery was very fast within 1day. Packaging ...
28,Singapore,Shopee-SG,5,2022-10-12 00:00:00,Performance: - '.' Best Feature(s): - '.' Valu...


In [120]:
final_output_df.shape

(411, 5)